In [ ]:
# default_exp normalization

# Normalization
The normalization step of Alphaquant is adapted from the [MS-EmpiRe algorithm](https://doi.org/10.1074/mcp.RA119.001509). It aims at reducing systematic biases between samples. Such biases can for example occur when more material is pipetted in one of the samples. In principle, two steps are performed:

1. Normalize between samples of the same condition (i.e. replicates)

2. Normalize between different conditions




## Within-condition normalization
It is common practice and highly recommended to measure multiple samples of a given condition. This ensures that observed changes between conditions are not just due to random variation. Examples of samples within the same condition could be biological replicates, but also patients with the same clinical condition. 
We want to ensure that systematic changes between within-condition samples are corrected for as follows:

* Our assumed input values are log2 transformed peptide ion intensities, which are stored in a 2d numpy array called "samples". Each row in samples represents a peptide and each column represents a sample

* In a first step, we determine the all pairwise distances between the samples (details explained below)
* We then choose the pair of samples with the closest distance between each other
* We randomly choose one "anchor" sample and one "shift" sample and we subtract the distance between the samples from each peptide intensity measured in the "shift" sample. This is equivalent to rescaling the intensities of the original sample by a constant factor such that the distributions are aligned
* We then construct a virtual "merged" sample by computing the average intensities of anchor and shift sample
* We repeat the steps above until all samples are merged. Keeping track of the shift factors allows us then to determine an ideal shift for each sample



In [ ]:
#hide
import alphaquant.norm.normalization as aqnorm

aqnorm.get_normfacts_withincond


### Find the best matching pair
Take all pairs of the columns in the "samples" array that have not been already merged and compute the distance between the pairs as follows:
* Subtract sample1 from sample2 (or sample2 from sample1, the order does not matter)
* This results in a distribution of differences. As the samples array contains log2 intensities, this corresponds to taking log2 fold changes
* Take the median of the distribution, this is a good approximation for the change between the two distributions
* Select the two samples with the lowest absolute change

In [ ]:
#hide

aqnorm.get_bestmatch_pair
aqnorm.create_distance_matrix


### Shifting samples
When we have computed the distance between two samples, we want to correct one of the samples by this distance. This results in two distributions with the same median value. We always shift the sample which has been merged from fewer distributions (see below for details). The sample to which the shift is applied is call "shift" sample and the sample which is not shifted is called "anchor" sample.
A "total shift" is calculated after all samples are merged, just by following up how many shifts have been applied to a sample in total

In [ ]:
#hide
aqnorm.shift_samples


### Merging distributions
After we shift two distributions on top of each other, we calculate a "merged" distribution. Each intensity in the merged distribution is the average of the intensity in both distributions. For the merging we have to take into account the following: If for example the anchor sample has already been merged from 10 samples, and the shift distribution has not been merged at all, we want to weigh the distribution coming from many samples higher. We hence multiply each sample by the number of merges.

In [ ]:
#hide
import time
import numpy as np

aqnorm.merge_distribs


## Compare Distributions

In [ ]:
#hide

aqnorm.determine_mode_iteratively
aqnorm.mode_normalization
aqnorm.get_betweencond_shift


## Wrapper functions

In [ ]:
#hide
aqnorm.normalize_if_specified


## Unit tests

In [ ]:
#hide
def test_merged_distribs():
    anchor_distrib = np.array([1, 1, 1, 1, 1])
    shift_distrib = np.array([2, 2, 2, 2, 2])
    counts_anchor_distrib = 4
    counts_shifted_distib = 1
    assert (aqnorm.merge_distribs(anchor_distrib, shift_distrib, counts_anchor_distrib, counts_shifted_distib)== np.array([1.2, 1.2, 1.2, 1.2, 1.2])).any()
    print("test_merged_distribs passed")

test_merged_distribs() 

In [ ]:
#hide
def generate_randarrays(number_arrays,size_of_array):
    randarray = []
    for i in range(number_arrays):
        shift = np.random.uniform(low=-10, high=+10)
        randarray.append(np.random.normal(loc=shift, size=size_of_array))
    return np.array(randarray)


In [ ]:
#hide
import numpy as np
import matplotlib.pyplot as plt

def test_sampleshift(samples):
    num_samples = samples.shape[0]
    merged_sample = []
    for i in range(num_samples):
        plt.hist(samples[i])
        merged_sample.extend(samples[i])
    stdev = np.std(merged_sample)
    print(f"STDev {stdev}")
    assert (stdev <=1.2) 
    
    plt.show()

randarray = generate_randarrays(5, 1000)
sample2shift = aqnorm.get_normfacts_withincond(randarray)
normalized_randarray = aqnorm.apply_sampleshifts(randarray, sample2shift)
test_sampleshift(normalized_randarray)